In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp meiosis

## meiosis
> Simulating Meisois and Recombination for various Crossing actions

~~ Note: might want to pull out the crossing over functions into a seperate module. They can be any function which returns a tensor of floats between 0 and max length of chromosome in cM 

In [ ]:
#| export
import torch
from chewc.core import *
from typing import Tuple, Optional, List, Union
import torch

def simulate_gametes(genome, parent_genomes, rate=1, shape=1, reps=1):
    """
    Simulate the formation of gametes for multiple parents using vectorized operations.

    Args:
        genome (Genome): The Genome instance containing the genetic map and other parameters.
        parent_genomes (torch.Tensor): Genomes of the parents.
                                       Shape: (num_individuals, ploidy, num_chromosomes, num_loci)
        rate (float): Rate parameter for the crossover model.
        shape (float): Shape parameter for the crossover model.
        reps (int): Number of repetitions to generate novel gametes.

    Returns:
        torch.Tensor: The resultant gametes.
                      Shape: (num_individuals, reps, ploidy//2, num_chromosomes, num_loci)
    """
    device = genome.device
    genetic_map = genome.genetic_map  # torch.Size([num_chromosomes, num_loci])
    num_individuals, ploidy, num_chromosomes, num_loci = parent_genomes.shape

    chromosome_lengths = genetic_map.max(dim=1).values

    # Simulate crossover positions for all chromosomes at once
    all_crossovers = poisson_crossing_over(chromosome_lengths)

    # Initialize gametes tensor with an additional dimension for repetitions
    gametes = torch.zeros(num_individuals, reps, ploidy // 2, num_chromosomes, num_loci, device=device, dtype=parent_genomes.dtype)

    for rep in range(reps):
        for chr_idx in range(num_chromosomes):
            crossovers = all_crossovers[chr_idx]

            if len(crossovers) > 0:
                crossover_mask = torch.zeros(num_loci, device=device, dtype=torch.bool)
                positions_idx = torch.searchsorted(genetic_map[chr_idx], crossovers)
                crossover_mask[positions_idx] = True

                parent_genome_1 = parent_genomes[:, ::2, chr_idx]
                parent_genome_2 = parent_genomes[:, 1::2, chr_idx]

                for ploid_idx in range(ploidy // 2):
                    gametes[:, rep, ploid_idx, chr_idx] = torch.where(crossover_mask.unsqueeze(0),
                                                                      parent_genome_1[:, ploid_idx],
                                                                      parent_genome_2[:, ploid_idx])
            else:
                gametes[:, rep, :, chr_idx] = parent_genomes[:, ::2, chr_idx]

    return gametes

# Define your Genome class or struct here if needed, ensuring it includes 'device' and 'genetic_map'

def poisson_crossing_over(chrom_lengths: torch.Tensor) -> list:
    """
    Generate crossing over locations for each chromosome given its length.

    Args:
    chrom_lengths (torch.Tensor): A 1D tensor containing the lengths of the chromosomes in centiMorgans.

    Returns:
    list of tensors: A list of tensors, each containing the crossing over locations for a chromosome.
    """
    # Assuming 'tensor' is your tensor and 'indices' are your indices
    if torch.cuda.is_available():
      device = torch.device('cuda')
    else:
      device = torch.device('cpu')
    crossing_over_locations = []
    for length in chrom_lengths:
        # Generate a random number of crossing over points for each chromosome
        num_crossovers = torch.randint(1, 5, (1,)).item() # Random number between 1 and 5
        # Generate random crossing over locations within the chromosome length
        crossovers = torch.rand(num_crossovers).to(device) * length
        # Sort the crossing over locations
        crossovers, _ = torch.sort(crossovers)
        crossing_over_locations.append(crossovers)

    return crossing_over_locations

In [ ]:
ploidy = 2
n_chr = 10
n_loci = 1000
n_Ind = 333
g = Genome(ploidy, n_chr, n_loci)
population = Population()
population.create_random_founder_population(g, n_founders=n_Ind)
init_pop = population.get_genotypes().float()  # gets allele dosage for calculating trait values

Created genetic map


In [ ]:
gametes = simulate_gametes(g,population.get_genotypes(), population, reps = 66) ; gametes.shape

torch.Size([333, 66, 1, 10, 1000])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()